In [3]:
! pip install tabulate

In [7]:
import pandas as pd
import sqlite3
from tabulate import tabulate  

# Load dataset
df = pd.read_csv("population_panel_clean.csv")

# Rename columns
df = df.rename(columns={
    "lifeexp": "LifeExpectancy",
    "u5mort": "U5Mortality",
    "adofert": "AdoleFertility"
})

# Create SQLite in-memory DB
conn = sqlite3.connect(":memory:")
df.to_sql("population", conn, index=False, if_exists="replace")

# SQL queries for descriptive statistics
queries = {
    "Total Rows": "SELECT COUNT(*) AS total_rows FROM population;",
    "Year Range": "SELECT MIN(year) AS start_year, MAX(year) AS end_year FROM population;",
    "Average Life Expectancy": "SELECT ROUND(AVG(LifeExpectancy), 2) AS avg_lifeexp FROM population;",
    "Average U5 Mortality": "SELECT ROUND(AVG(U5Mortality), 2) AS avg_u5mort FROM population;",
    "Average Adolescent Fertility": "SELECT ROUND(AVG(AdoleFertility), 2) AS avg_fertility FROM population;",
    "Number of Countries": "SELECT COUNT(DISTINCT country_name) AS total_countries FROM population;",
    "Average Years per Country": """
        SELECT ROUND(AVG(entry_count), 2) AS avg_years_per_country
        FROM (
            SELECT country_name, COUNT(*) AS entry_count
            FROM population
            GROUP BY country_name
        );
    """
}

# Execute queries and collect results
summary_table = []
for label, query in queries.items():
    result = pd.read_sql(query, conn)
    value = result.iloc[0, 0]
    summary_table.append([label, value])

# Print results in table format
print(tabulate(summary_table, headers=["Statistic", "Value"], tablefmt="fancy_grid"))

# Close the DB connection
conn.close()



╒══════════════════════════════╤══════════╕
│ Statistic                    │    Value │
╞══════════════════════════════╪══════════╡
│ Total Rows                   │ 11536    │
├──────────────────────────────┼──────────┤
│ Year Range                   │  1960    │
├──────────────────────────────┼──────────┤
│ Average Life Expectancy      │    64.93 │
├──────────────────────────────┼──────────┤
│ Average U5 Mortality         │    73.06 │
├──────────────────────────────┼──────────┤
│ Average Adolescent Fertility │    73.58 │
├──────────────────────────────┼──────────┤
│ Number of Countries          │   195    │
├──────────────────────────────┼──────────┤
│ Average Years per Country    │    59.16 │
╘══════════════════════════════╧══════════╛
